# Bayesian Hierarchical Marketing Mix Modeling in PyMC

- https://towardsdatascience.com/bayesian-hierarchical-marketing-mix-modeling-in-pymc-684f6024e57a

In [7]:
import pandas as pd
import theano.tensor as tt


WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [8]:
data = pd.read_csv(   'https://raw.githubusercontent.com/Garve/datasets/4576d323bf2b66c906d5130d686245ad205505cf/mmm.csv',
    parse_dates=['Date'],
    index_col='Date'
)
X = data.drop(columns=['Sales'])
y = data['Sales']
data.head()

,TV,Radio,Banners,Sales
Date,,,,
2018-01-07,13528.1,0.00,0.00,9779.80
2018-01-14,0.0,5349.65,2218.93,13245.19
2018-01-21,0.0,4235.86,2046.96,12022.66
2018-01-28,0.0,3562.21,0.00,8846.95
2018-02-04,0.0,0.00,2187.29,9797.07


In [6]:
!pip install theano

  Using cached Theano-1.0.5.tar.gz (2.8 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for theano: filename=Theano-1.0.5-py3-none-any.whl size=2668122 sha256=a79ff5946cd187a4c8e710093ca09e51bf0e50091380dd79b45156f052dd4cf9
  Stored in directory: c:\users\a1274500\appdata\local\pip\cache\wheels\84\cb\19\235b5b10d89b4621f685112f8762681570a9fa14dc1ce904d9
Successfully built theano


In [9]:
# 
def saturate(x, a):
    return 1 - tt.exp(-a*x)

def carryover(x, strength, length=21):
    w = tt.as_tensor_variable(
        [tt.power(strength, i) for i in range(length)]
    )
    
    x_lags = tt.stack(
        [tt.concatenate([
            tt.zeros(i),
            x[:x.shape[0]-i]
        ]) for i in range(length)]
    )
    
    return tt.dot(w, x_lags)